# Data Cleaning

In [62]:
import pandas as pd

# Load the NYPD dataset
nypd_data = pd.read_csv("/Users/shashankhmg/Documents/AXA-Casestudy/Data-Science-Challenge/data/raw/nypd/Motor_Vehicle_Collisions_-_Crashes_20250218.csv")

/var/folders/tw/yc01l_99601d14ks8nxwg1dr0000gn/T/ipykernel_88113/3332287003.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  nypd_data = pd.read_csv("/Users/shashankhmg/Documents/AXA-Casestudy/Data-Science-Challenge/data/raw/nypd/Motor_Vehicle_Collisions_-_Crashes_20250218.csv")


In [63]:
nypd_data.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,11/01/2023,1:29,BROOKLYN,11230.0,40.62179,-73.970024,"(40.62179, -73.970024)",OCEAN PARKWAY,AVENUE K,NaN,...,Unspecified,Unspecified,NaN,NaN,4675373,Moped,Sedan,Sedan,NaN,NaN
3,06/29/2022,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
4,09/21/2022,13:21,NaN,NaN,NaN,NaN,NaN,BROOKLYN BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4566131,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN


In [64]:
nypd_data.isnull().sum()


CRASH DATE                             0
CRASH TIME                             0
BOROUGH                           667542
ZIP CODE                          667810
LATITUDE                          239669
LONGITUDE                         239669
LOCATION                          239669
ON STREET NAME                    463881
CROSS STREET NAME                 822431
OFF STREET NAME                  1784849
NUMBER OF PERSONS INJURED             18
NUMBER OF PERSONS KILLED              31
NUMBER OF PEDESTRIANS INJURED          0
NUMBER OF PEDESTRIANS KILLED           0
NUMBER OF CYCLIST INJURED              0
NUMBER OF CYCLIST KILLED               0
NUMBER OF MOTORIST INJURED             0
NUMBER OF MOTORIST KILLED              0
CONTRIBUTING FACTOR VEHICLE 1       7385
CONTRIBUTING FACTOR VEHICLE 2     341236
CONTRIBUTING FACTOR VEHICLE 3    2000923
CONTRIBUTING FACTOR VEHICLE 4    2120972
CONTRIBUTING FACTOR VEHICLE 5    2146690
COLLISION_ID                           0
VEHICLE TYPE COD

In [65]:
nypd_data.columns

Index(['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME',
       'OFF STREET NAME', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1',
       'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
       'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
       'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2',
       'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'],
      dtype='object')

In [66]:
#Some columns have extreme levels of missing values (>80%), making them less useful for analysis.
nypd_data_cleaned = nypd_data.drop(columns=['CROSS STREET NAME','OFF STREET NAME',
                                            'CONTRIBUTING FACTOR VEHICLE 3','CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
                                            'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'])


In [67]:
# Drop rows where LATITUDE & LONGITUDE are both missing
nypd_data_cleaned = nypd_data_cleaned.dropna(subset=['LATITUDE', 'LONGITUDE'])


In [68]:
# Fill missing values using borough-based median coordinates (if borough is known)
#ie. if we have the lat and long for one of the observations of borough and the others are null, then we can impute using the same. 
nypd_data_cleaned['LATITUDE'] = nypd_data_cleaned.groupby('BOROUGH')['LATITUDE'].transform(lambda x: x.fillna(x.median()))
nypd_data_cleaned['LONGITUDE'] = nypd_data_cleaned.groupby('BOROUGH')['LONGITUDE'].transform(lambda x: x.fillna(x.median()))


In [69]:
# Define columns that need to be filled with "Unknown"
fill_unknown_cols = [
    'BOROUGH', 'ZIP CODE', 'ON STREET NAME', 
    'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2',
    'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2'
]

# Fill missing values for these columns
nypd_data_cleaned[fill_unknown_cols] = nypd_data_cleaned[fill_unknown_cols].fillna("Unknown")


In [70]:
# columns are necessary because it is Needed for risk assessment
injury_cols = ['NUMBER OF PERSONS INJURED','NUMBER OF PERSONS KILLED',
               'NUMBER OF CYCLIST INJURED','NUMBER OF CYCLIST KILLED',
               'NUMBER OF MOTORIST INJURED','NUMBER OF MOTORIST KILLED']
nypd_data_cleaned.loc[:,injury_cols] = nypd_data_cleaned[injury_cols].fillna(0).astype(int)

In [71]:
# Define NYC's approximate lat/lng boundaries
NYC_LAT_MIN, NYC_LAT_MAX = 40.5, 40.9
NYC_LNG_MIN, NYC_LNG_MAX = -74.2, -73.7

# Filter valid geolocation data
nypd_data_cleaned = nypd_data_cleaned[
    (nypd_data_cleaned['LATITUDE'].between(NYC_LAT_MIN, NYC_LAT_MAX)) &
    (nypd_data_cleaned['LONGITUDE'].between(NYC_LNG_MIN, NYC_LNG_MAX))
]


In [72]:
# Check if missing values remain
nypd_data_cleaned.isnull().sum()


CRASH DATE                       0
CRASH TIME                       0
BOROUGH                          0
ZIP CODE                         0
LATITUDE                         0
LONGITUDE                        0
LOCATION                         0
ON STREET NAME                   0
NUMBER OF PERSONS INJURED        0
NUMBER OF PERSONS KILLED         0
NUMBER OF PEDESTRIANS INJURED    0
NUMBER OF PEDESTRIANS KILLED     0
NUMBER OF CYCLIST INJURED        0
NUMBER OF CYCLIST KILLED         0
NUMBER OF MOTORIST INJURED       0
NUMBER OF MOTORIST KILLED        0
CONTRIBUTING FACTOR VEHICLE 1    0
CONTRIBUTING FACTOR VEHICLE 2    0
COLLISION_ID                     0
VEHICLE TYPE CODE 1              0
VEHICLE TYPE CODE 2              0
dtype: int64

In [74]:
print(nypd_data_cleaned.shape)
print(nypd_data.shape)

(1437817, 21)
(2156357, 29)


In [75]:
# Save as CSV
nypd_data_cleaned.to_csv(r"/Users/shashankhmg/Documents/AXA-Casestudy/Data-Science-Challenge/data/processed/nypd_data_cleaned.csv", index=False)

# EDA

In [79]:
#Visualizing High-Risk Accident Zones (Geospatial Heatmap)
# Create an Accident Heatmap Using Folium

import folium
import pandas as pd
from folium.plugins import HeatMap

# Filter for accidents involving cyclists
cyclist_accidents = nypd_data_cleaned[
    (nypd_data_cleaned['NUMBER OF CYCLIST INJURED'] > 0) | 
    (nypd_data_cleaned['NUMBER OF CYCLIST KILLED'] > 0)
]

# Create base map centered around NYC
nyc_map = folium.Map(location=[40.75, -73.98], zoom_start=12)

# Extract accident locations for heatmap
heatmap_data = cyclist_accidents[['LATITUDE', 'LONGITUDE']].dropna().values.tolist()

# Add heatmap layer
HeatMap(heatmap_data, radius=10, blur=15, max_zoom=1).add_to(nyc_map)

# Save the heatmap
nyc_map.save("cyclist_accident_heatmap.html")
print(" Heatmap saved as 'cyclist_accident_heatmap.html'")


 Heatmap saved as 'cyclist_accident_heatmap.html'
